# Breast Cancer Detection

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
from scipy import stats
from sklearn.model_selection import train_test_split

### NOTE:  
Thuật toán K-Nearest Neighbors là một thuật toán phân loại mạnh mẽ nhưng đơn giản. Nó đưa ra dự đoán bằng cách so sánh các điểm dữ liệu mới với k điểm dữ liệu gần nhất và gán lớp dựa trên lớp chiếm đa số của các điểm láng giềng đó.

Lưu ý rằng không có giai đoạn huấn luyện như trong các thuật toán học máy khác. Thay vào đó, tập dữ liệu huấn luyện chỉ được lưu lại để sử dụng trong giai đoạn dự đoán sau này.

### Một vài vấn đề khi chọn K:  
Hiệu suất của thuật toán có thể nhạy cảm với giá trị của k. Khi k nhỏ, mô hình nhạy cảm với nhiễu và dễ bị overfitting, trong khi các giá trị k lớn có thể dẫn đến underfitting, đặc biệt nếu có sự mất cân bằng giữa các lớp.

Khi số lượng lớp là 2, k nên là một số lẻ để tránh tình trạng "bỏ phiếu hòa" khi đưa ra dự đoán.

Giá trị của k nên lớn hơn số lượng lớp vì những lý do tương tự.

### Tập dữ liệu:
Tập dữ liệu chúng ta làm việc lần này là UCI breast cancer dataset (có sẵn link trên kaggle). Bài toán ta cần giải quyết là classification để dự đoán 1 người có mắc ung thư vú hay không

In [ ]:
# Load data
data = pd.read_csv('data.csv',index_col='id')

In [ ]:
data.reset_index(drop=True, inplace=True)

#### Basic EDA

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.drop(['Unnamed: 32'],axis=1,inplace=True)

In [ ]:
sns.heatmap(data.corr(numeric_only=True))
plt.show()

In [ ]:
# Giả sử bạn có DataFrame df và muốn vẽ phân phối cho các cột số
numeric_columns = data.select_dtypes(include='number').columns  # Chọn các cột số trong DataFrame

# Số lượng cột số
n = len(numeric_columns)

# Thiết lập số dòng và cột cho lưới vẽ
fig, axes = plt.subplots(nrows=(n // 3) + (n % 3), ncols=3, figsize=(15, 5 * ((n // 3) + (n % 3))))

# Làm phẳng các axes
axes = axes.flatten()

# Vẽ phân phối cho mỗi cột số
for i, col in enumerate(numeric_columns):
    axes[i].hist(data[col], bins=30, alpha=0.7)
    axes[i].set_title(f'Phân phối của {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Tần suất')

# Ẩn đi các axes thừa
for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
plt.pie(data["diagnosis"].value_counts(), labels=["Benign", "Malignant"], autopct='%1.1f%%')
plt.title("Tỷ lệ phân loại của biến mục tiêu")
plt.show()

#### Pre-processing data

In [ ]:
X = data.drop('diagnosis', axis=1)
y = data['diagnosis']

# Encode label
y = y.map({'M': 1, 'B': 0})

In [ ]:
y.value_counts()

Do tập dữ liệu đã được làm sạch từ trước đó nên công việc làm sạch ta bỏ qua

In [ ]:
## Split data 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Xây dựng model KNN

In [ ]:
class kNN():
    '''k-Nearest Neighbours'''
    # Initialise
    def __init__(self, k=3, metric='euclidean', p=None):
        self.k = k
        self.metric = metric
        self.p = p
    
    # Euclidean distance (l2 norm)
    def euclidean(self, v1, v2):
        return np.sqrt(np.sum((v1-v2)**2))
    
    # Manhattan distance (l1 norm)
    def manhattan(self, v1, v2):
        return np.sum(np.abs(v1-v2))
    
    # Minkowski distance (lp norm)
    def minkowski(self, v1, v2, p=2):
        return np.sum(np.abs(v1-v2)**p)**(1/p)
        
    # Store train set
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    # Make predictions
    def predict(self, X_test):
        preds = []
        # Loop over rows in test set
        for test_row in X_test:
            nearest_neighbours = self.get_neighbours(test_row)
            majority = stats.mode(nearest_neighbours)[0][0]
            preds.append(majority)
        return np.array(preds)
    
    # Get nearest neighbours
    def get_neighbours(self, test_row):
        distances = list()
        
        # Calculate distance to all points in X_train
        for (train_row, train_class) in zip(self.X_train, self.y_train):
            if self.metric=='euclidean':
                dist = self.euclidean(train_row, test_row)
            elif self.metric=='manhattan':
                dist = self.manhattan(train_row, test_row)
            elif self.metric=='minkowski':
                dist = self.minkowski(train_row, test_row, self.p)
            else:
                raise NameError('Supported metrics are euclidean, manhattan and minkowski')
            distances.append((dist, train_class))
            
        # Sort distances
        distances.sort(key=lambda x: x[0])
        
        # Identify k nearest neighbours
        neighbours = list()
        for i in range(self.k):
            neighbours.append(distances[i][1])
            
        return neighbours

In [ ]:
X_test.reset_index(drop=True, inplace=True)

In [ ]:
def accuracy(preds, y_test):
    return 100 * (preds == y_test).mean()

clf = kNN(k=5,metric='euclidean')
clf.fit(X_train.values,y_train.values)
preds = clf.predict(X_test.values)

print(f"Accuracy: {accuracy(preds, y_test)}")